In [ ]:
cd ..

In [ ]:
import echofilter.raw

In [ ]:
import copy
import os

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
turbulence_color = "c"
bottom_color = "#00dd00"

In [ ]:
root_data_dir = "/media/scott/scratch/Datasets/dsforce/surveyExports"

# example with only passive period
# sample = 'mobile/Survey17/Survey17_GR4_T1W_E'

# example with 1 passive period, 1 turbulence cut out
# sample = 'mobile/Survey16/Survey16_GR3_N1W_E'

# example with lots of short passive periods, 1 proper passive period, 1 turbulence cut out
# sample = 'mobile/Survey17/Survey17_GR1_N0W_E'

# example with passive, removed, and patches
sample = "mobile/Survey16/Survey16_GR1_N3A_F"

# example with passive, removed, and patches
sample = "mobile/Survey16/Survey16_GR3_N3A_F"

# example where passive detection went wrong
# sample = 'MinasPassage/march2018/march2018_D20180330-T202218_D20180331-T015214'

# Load raw data
fname_raw = os.path.join(root_data_dir, sample + "_Sv_raw.csv")
fname_masked = os.path.join(root_data_dir, sample + "_Sv.csv")

ts_raw, depths_raw, signals_raw = echofilter.raw.loader.transect_loader(fname_raw)
ts_mskd, depths_mskd, signals_mskd = echofilter.raw.loader.transect_loader(fname_masked)
mask = ~np.isnan(signals_mskd)

fname_top1 = os.path.join(root_data_dir, sample + "_turbulence.evl")
fname_top2 = os.path.join(root_data_dir, sample + "_air.evl")
fname_bot = os.path.join(root_data_dir, sample + "_bottom.evl")
if os.path.isfile(fname_top1):
    t_top, d_top = echofilter.raw.loader.evl_loader(fname_top1)
elif os.path.isfile(fname_top2):
    t_top, d_top = echofilter.raw.loader.evl_loader(fname_top2)
else:
    t_top = d_top = None
if os.path.isfile(fname_bot):
    t_bot, d_bot = echofilter.raw.loader.evl_loader(fname_bot)
else:
    t_bot = d_bot = None

In [ ]:
(
    d_top_new,
    d_bot_new,
    passive_starts,
    passive_ends,
) = echofilter.raw.manipulate.fixup_lines(
    ts_raw,
    depths_raw,
    signals_raw,
    mask,
    t_top=t_top,
    d_top=d_top,
    t_bot=t_bot,
    d_bot=d_bot,
    return_passive_boundaries=True,
)
ts_new = ts_raw

In [ ]:
t_top[-1] - t_top[0]

In [ ]:
ts_raw[-1] - ts_raw[0]

In [ ]:
ts_mskd[-1] - ts_mskd[0]

In [ ]:
plt.plot(t_top, d_top)

In [ ]:
is_passive = np.zeros(ts_raw.shape, dtype=bool)

for pass_start, pass_end in zip(passive_starts, passive_ends):
    is_passive[pass_start:pass_end] = True

In [ ]:
allnan = np.all(np.isnan(signals_mskd), axis=1)

is_removed = allnan & ~is_passive

removed_starts = np.nonzero(np.diff(is_removed.astype(np.float)) > 0)[0]
removed_ends = np.nonzero(np.diff(is_removed.astype(np.float)) < 0)[0]

if len(removed_starts) > 0:
    removed_starts += 1
if len(removed_ends) > 0:
    removed_ends += 1

if len(removed_ends) > 0 and (
    len(removed_starts) == 0 or removed_ends[0] < removed_starts[0]
):
    removed_starts = np.concatenate(([0], removed_starts))

if len(removed_starts) > 0 and (
    len(removed_ends) == 0 or removed_starts[-1] > removed_ends[-1]
):
    removed_ends = np.concatenate((removed_ends, [len(is_removed)]))

In [ ]:
# Plot with time on x-axis
for ts, depths, signals in (
    (ts_raw, depths_raw, signals_raw),
    (ts_mskd, depths_mskd, signals_mskd),
):
    plt.figure(figsize=(12, 12))
    plt.pcolormesh(ts, depths, signals.T)
    if d_top is not None:
        li = t_top <= ts[-1]
        eliminated_line_points = np.sum(~li)
        if eliminated_line_points > 1:
            print(
                "Removed {} point from top line which extend past signal recording".format(
                    eliminated_line_points
                )
            )
        plt.plot(t_top[li], d_top[li], turbulence_color)
    if d_bot is not None:
        plt.plot(t_bot, d_bot, bottom_color)
    plt.gca().invert_yaxis()
    plt.xlabel("Timestamp (s)")
    plt.ylabel("Depth (m)")
    plt.show()

In [ ]:
# Plot with index on x-axis
for ts, depths, signals in (
    (np.arange(signals_raw.shape[0]), depths_raw, signals_raw),
    (np.arange(signals_raw.shape[0]), depths_mskd, signals_mskd),
):
    plt.figure(figsize=(12, 12))
    plt.pcolormesh(ts, depths, signals.T)
    if d_top is not None:
        plt.plot(ts, np.interp(ts_raw, t_top, d_top), turbulence_color)
    if d_bot is not None:
        plt.plot(ts, np.interp(ts_raw, t_bot, d_bot), bottom_color)
    plt.gca().invert_yaxis()
    plt.xlabel("Sample index")
    plt.ylabel("Depth (m)")
    plt.show()

In [ ]:
bad_mask = np.ones(signals_raw.shape, dtype=bool)

ddepths = np.broadcast_to(depths_raw, signals_raw.shape)

is_top = ddepths < np.expand_dims(np.interp(ts_raw, t_top, d_top), -1)
bad_mask[is_top] = False

if d_bot is not None:
    is_bot = ddepths > np.expand_dims(np.interp(ts_raw, t_bot, d_bot), -1)
    bad_mask[is_bot] = False

signals_badly = copy.deepcopy(signals_raw)
signals_badly[~bad_mask] = np.NaN

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_raw, depths_raw, signals_badly.T)
plt.plot(t_top, d_top, turbulence_color)
plt.plot(t_bot, d_bot, bottom_color)
plt.gca().invert_yaxis()
plt.xlabel("Timestamp (s)")
plt.ylabel("Depth (m)")
plt.title("Bad mask (old)")
plt.show()

In [ ]:
for ts, depths, signals in (
    (ts_raw, depths_raw, signals_raw),
    (ts_mskd, depths_mskd, signals_mskd),
):
    plt.figure(figsize=(12, 12))
    plt.pcolormesh(ts, depths, signals.T)
    plt.plot(ts_new, d_top_new, turbulence_color)
    plt.plot(ts_new, d_bot_new, bottom_color)

    for r_start, r_end in zip(passive_starts, passive_ends):
        plt.fill_between(
            ts_raw[[r_start, r_end]],
            depths_raw[[0, 0]],
            depths_raw[[-1, -1]],
            facecolor="none",
            hatch="//",
            edgecolor="k",
            linewidth=0.0,
        )

    for r_start, r_end in zip(removed_starts, removed_ends):
        plt.fill_between(
            ts_raw[[r_start, r_end]],
            depths_raw[[0, 0]],
            depths_raw[[-1, -1]],
            facecolor="none",
            hatch="\\\\",
            edgecolor=[0, 0, 1],
            linewidth=0.0,
        )

    plt.xlabel("Timestamp (s)")
    plt.ylabel("Depth (m)")
    plt.gca().invert_yaxis()
    plt.show()

## Recompose mask from parts

In [ ]:
new_mask = np.ones(signals_raw.shape, dtype=bool)

new_mask[is_passive, :] = False
new_mask[is_removed, :] = False

ddepths = np.broadcast_to(depths_raw, signals_raw.shape)
is_top = ddepths < np.expand_dims(np.nan_to_num(d_top_new), -1)
is_bot = ddepths > np.expand_dims(np.nan_to_num(d_bot_new), -1)

new_mask[is_top] = False
new_mask[is_bot] = False

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_raw, depths_raw, signals_raw.T)
plt.gca().invert_yaxis()
plt.xlabel("Timestamp (s)")
plt.ylabel("Depth (m)")
plt.title("Raw data")
plt.show()

plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_raw, depths_raw, signals_mskd.T)
plt.gca().invert_yaxis()
plt.xlabel("Timestamp (s)")
plt.ylabel("Depth (m)")
plt.title("Masked data")
plt.show()

plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_raw, depths_raw, mask.T)
plt.gca().invert_yaxis()
plt.xlabel("Timestamp (s)")
plt.ylabel("Depth (m)")
plt.title("Original mask")
plt.show()

plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_raw, depths_raw, new_mask.T)
plt.gca().invert_yaxis()
plt.xlabel("Timestamp (s)")
plt.ylabel("Depth (m)")
plt.title("Decomposed and recomposed mask")
plt.show()

### Check recomposed mask matches the original mask

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_raw, depths_raw, (np.single(mask) - np.single(new_mask)).T)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
np.nonzero(np.single(mask) - np.single(new_mask))

In [ ]:
np.max(np.single(mask) - np.single(new_mask))

In [ ]:
np.min(np.single(mask) - np.single(new_mask))

In [ ]:
np.unique(np.nonzero(np.single(mask) - np.single(new_mask))[0])

In [ ]:
for idx in np.unique(np.nonzero(np.single(mask) - np.single(new_mask))[0]):
    for data in (signals_raw, mask, new_mask):
        plt.figure(figsize=(12, 12))
        plt.pcolormesh(
            ts_raw[max(0, idx - 4) : min(len(ts_raw), idx + 5)],
            depths_raw[:80],
            data[max(0, idx - 4) : min(len(ts_raw), idx + 5), :80].T,
        )
        plt.gca().invert_yaxis()
        plt.show()

### Visually inspect starts and ends of removed segments

In [ ]:
for idx in np.concatenate((passive_starts, passive_ends, removed_starts, removed_ends)):
    for data in (signals_raw, mask, new_mask):
        plt.figure(figsize=(12, 12))
        plt.pcolormesh(
            ts_raw[idx - 4 : idx + 5], depths_raw[:500], data[idx - 4 : idx + 5, :500].T
        )
        plt.gca().invert_yaxis()
        plt.show()